In [1]:
!pip install -U unsloth rouge-score bert-score bleu evaluate datasets torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
import random, torch
from datasets import load_dataset, Dataset # Added Dataset to imports
import evaluate
import json # Added json to imports
import os # Added os for file existence checks

def load_cleaned_jsonl_dataset(file_path):
    """
    Loads a JSON Lines dataset from a file, skipping malformed or empty lines.
    Each valid line is parsed as a JSON object.
    """
    data_dicts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            line = line.strip()
            if not line: # Skip empty lines
                continue
            try:
                obj = json.loads(line)
                if obj is not None: # Skip lines that are just 'null'
                    data_dicts.append(obj)
            except json.JSONDecodeError as e:
                # Log the error for the user to see, but continue processing
                print(f"Skipping malformed JSON line {i+1} in {file_path}: {line}. Error: {e}")
            except Exception as e:
                print(f"Skipping line {i+1} due to unexpected error: {e}")

    if not data_dicts:
        raise ValueError(f"No valid JSON lines found in {file_path} after cleaning.")

    # Create a Dataset object from the list of dictionaries
    return Dataset.from_list(data_dicts)


In [4]:
DEVICE = "cuda"
results = []
done = set()
PREDICTIONS_FILE = "ft2-predictions.jsonl"

In [5]:
# ---------------- CONFIG ----------------
MAX_NEW_TOKENS = 256
BASE_MODEL = "unsloth/mistral-7b-bnb-4bit"
ADAPTER_REPO = "devanshpursnani/mistral7b-ectsum2"

# ---------------- MODEL -----------------
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=4096,
    dtype=torch.float16,
    load_in_4bit=True,
    device_map={"": 0},   # <-- force everything on GPU (fix)
)

# attach LoRA structure (must match training)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=8,
    lora_dropout=0.0,
    bias="none",
)

# load trained adapters
model.load_adapter(ADAPTER_REPO, adapter_name=ADAPTER_REPO)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2026.1.2: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2026.1.2 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                  (devanshpursnani/mistral7b-ectsum2): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                  (devanshpursnani/mistral7b-ectsum2): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                  (dev

In [6]:
import json

TEST_FILE = "test500.jsonl"

print("Loading test500.jsonl ...")

ds = load_cleaned_jsonl_dataset(TEST_FILE)

inputs = [r["input"] for r in ds]
refs   = [r["output"] for r in ds]

print(f"Loaded {len(ds)} samples from {TEST_FILE}")

Loading test500.jsonl ...
Loaded 500 samples from test500.jsonl


In [7]:
# --------- BATCH CONTROL ----------
BATCH_START = 0      # first run: 0
BATCH_END   = 300    # first run: 300

# second run:
# BATCH_START = 300
# BATCH_END   = 500

batch_inputs = inputs[BATCH_START:BATCH_END]
batch_refs   = refs[BATCH_START:BATCH_END]

In [8]:
import tqdm

for j, prompt in enumerate(tqdm.tqdm(batch_inputs, desc="Generating")):
    i = BATCH_START + j   # real index in full dataset

    batch = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=4096,
    ).to(DEVICE)

    with torch.no_grad():
        out = model.generate(
            **batch,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
        )

    text = tokenizer.decode(out[0], skip_special_tokens=True)
    if "<EXEC_SUMMARY>" in text:
        text = text.split("<EXEC_SUMMARY>")[-1].strip()

    row = {
        "id": i,
        "input": prompt,
        "reference": batch_refs[j],
        "prediction": text,
    }

    results.append(row)


PREDICTIONS_FILE = "preds_part1.jsonl"

with open(PREDICTIONS_FILE, "w", encoding="utf-8") as f:
    for r in results:
        json.dump(r, f, ensure_ascii=False)
        f.write("\n")



print("file saved")



Generating: 100%|██████████| 300/300 [1:09:00<00:00, 13.80s/it]

file saved


In [ ]:
import json
import evaluate

PREDICTIONS_FILE = "predictions.jsonl"   # use your actual path

# --------- LOAD FROM FILE ----------
preds = []
refs_ = []

with open(PREDICTIONS_FILE, "r", encoding="utf-8") as f:
    for line in f:
        r = json.loads(line)
        preds.append(r["prediction"])
        refs_.append(r["reference"])

print(f"Loaded {len(preds)} predictions from file")

# --------- METRICS ----------
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

r = rouge.compute(predictions=preds, references=refs_)
b = bleu.compute(predictions=preds, references=[[x] for x in refs_])
bs = bertscore.compute(predictions=preds, references=refs_, lang="en")

print("\n=== METRICS ===")
print("ROUGE-1:", r["rouge1"])
print("ROUGE-2:", r["rouge2"])
print("ROUGE-L:", r["rougeL"])
print("BLEU:", b["bleu"])
print("BERTScore F1:", sum(bs["f1"]) / len(bs["f1"]))